In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
import pandas as pd
import numpy as np
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [2]:
def encode(text):
    
    
    marked_text = "[CLS] " + text + " [SEP]"

#     print (marked_text)

    tokenized_text = tokenizer.tokenize(marked_text)
#     print (tokenized_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

#     for tup in zip(tokenized_text, indexed_tokens):
#         print (tup)

    segments_ids = [1] * len(tokenized_text)
#     print (segments_ids)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

   


    # Predict hidden states features for each layer
    with torch.no_grad():
        encoded_layers, _ = model(tokens_tensor, segments_tensors)
    sentence_embedding = torch.mean(encoded_layers[11], 1)
    
    return sentence_embedding

In [3]:
data = pd.read_csv('/home/ashutosh/faq_chatbot.csv')
questions = list(data['Questions'])

In [4]:
doc_vecs=[]
for i in range(0,len(questions)):
    
    doc_vecs.append(np.array( encode(questions[i])))

In [5]:
encoded_array=np.concatenate(doc_vecs)

In [6]:
topk =4
while True:
    query = input('your question: ')
    query_vec = np.array( encode(query))
    # compute normalized dot product as score
    score = np.sum(query_vec * encoded_array, axis=1) / np.linalg.norm(encoded_array, axis=1)
    topk_idx = np.argsort(score)[::-1][:topk]
    for idx in topk_idx:
        print('> %s\t%s' % (score[idx], questions[idx]))

your question: my name is what
> 5.365883	I need to know Security guard number?
> 5.3024383	I lost my belonging. . whom should i approach?
> 5.2852407	What is the email address of IT  HR Admin team
> 5.252398	i need to know the Emergency contact number
your question: security number
> 6.059444	I need to know Security guard number?
> 5.9243393	i need to know the Emergency contact number
> 5.376879	Please provide me extension number for IT  HR Admin team
> 5.336084	i need to know my parking slot. please provide me contact details. 
your question: contact nuber
> 4.919416	What is the email address of IT  HR Admin team
> 4.844924	Please provide me extension number for IT  HR Admin team
> 4.823474	how to reach to IT HR Admin and so?
> 4.741486	i need to know the Emergency contact number
your question: contact number
> 6.652817	i need to know the Emergency contact number
> 6.5223656	i need to know my parking slot. please provide me contact details. 
> 6.5060024	I have a complaint about someo

KeyboardInterrupt: 